In [1]:
# Import key librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [2]:
# Setup Chart Studio
username = 'mrcrookes' # your username
api_key = 'ZgEVmRwdEN7u8cqkga9A' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [42]:
# Define level
level = 'Advanced_Higher'
level_label = level.replace('_', ' ')

# Import data
ind_df = pd.read_excel("attainment-statistics-(august)-2022-provisional-centre-independent.ods", 
                       engine = "odf", sheet_name = level, skiprows = 2)

state_df = pd.read_excel("attainment-statistics-(august)-2022-provisional-centre-education-authority.ods", 
                        engine = "odf", sheet_name = level, skiprows = 2)

In [43]:
def tidy_up_1(df):
    
    # Remove spaces and replace with '_'
    df.columns = df.columns.str.replace(' ','_')

    #df.drop(df[df.Grade_A_Count_2022 == '[z]'].index, inplace=True)
    df = df.replace('[z]', np.nan)
    df = df.replace('[c]', np.nan)
    df = df.replace('[low]', np.nan)

    # Remove last row
    df = df.drop(df.index[-1])

    # Remove columns with Percentage in the heading
    df = df[df.columns.drop(list(df.filter(regex='Percentage')))]
    
    return df
    
ind_df = tidy_up_1(ind_df)
state_df = tidy_up_1(state_df)

In [44]:
def tidy_up_2(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018']

    # Sort out column titles
    for year in years:
        df = df.rename({'Grade_A_Count_' + year: 'As_' + year}, axis=1)
        df = df.rename({'Grades_A-B_Count_' + year: 'A-Bs_' + year}, axis=1)
        df = df.rename({'Grades_A-C_Count_' + year: 'A-Cs_' + year}, axis=1)
        df = df.rename({'Grades_A-D_Count_' + year: 'A-Ds_' + year}, axis=1)
        df = df.rename({'No_Award_Count_' + year: 'NAs_' + year}, axis=1)

    # Calculate number of Bs, Cs and Ds
    for year in years:
        df['Bs_' + year] = df['A-Bs_' + year] - df['As_' + year]
        df['Cs_' + year] = df['A-Cs_' + year] - df['A-Bs_' + year]
        df['Ds_' + year] = df['A-Ds_' + year] - df['A-Cs_' + year]

    # Remove original columns
    for year in years:
        df.drop('A-Bs_' + year, axis=1, inplace=True)
        df.drop('A-Cs_' + year, axis=1, inplace=True)
        df.drop('A-Ds_' + year, axis=1, inplace=True)
        
    return df
        
ind_df = tidy_up_2(ind_df)
state_df = tidy_up_2(state_df)

In [45]:
ind_df.head()

,Subject,As_2022,NAs_2022,Entries_2022,As_2021,NAs_2021,Entries_2021,As_2020,NAs_2020,Entries_2020,...,Ds_2021,Bs_2020,Cs_2020,Ds_2020,Bs_2019,Cs_2019,Ds_2019,Bs_2018,Cs_2018,Ds_2018
0,Accounting,10.0,0.0,15.0,20.0,0.0,25.0,15.0,0.0,20.0,...,0.0,0.0,5.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0
1,Art and Design (Design),40.0,0.0,60.0,35.0,0.0,45.0,35.0,0.0,45.0,...,0.0,5.0,5.0,0.0,15.0,5.0,0.0,15.0,10.0,0.0
2,Art and Design (Expressive),65.0,0.0,95.0,65.0,0.0,85.0,75.0,0.0,95.0,...,0.0,15.0,5.0,0.0,20.0,5.0,0.0,10.0,10.0,0.0
3,Biology,200.0,15.0,465.0,320.0,5.0,485.0,250.0,NaN,430.0,...,15.0,115.0,55.0,10.0,95.0,65.0,30.0,110.0,75.0,20.0
4,Business Management,90.0,10.0,240.0,110.0,NaN,165.0,120.0,0.0,180.0,...,5.0,35.0,25.0,0.0,30.0,30.0,5.0,40.0,25.0,10.0


In [46]:
# Convert into long format

def tolong(df):
    ldf = pd.melt(df, id_vars = ['Subject'], var_name = 'abc', value_name ='Count')
    ldf['Grades'] = ldf['abc'].apply(lambda x: (x.rsplit('_', 1))[0])
    ldf['Year'] = ldf.pop('abc').apply(lambda x: (x.split('_'))[-1])
    
    return ldf
    
ind_ldf = tolong(ind_df)
state_ldf = tolong(state_df)

state_ldf.head()

,Subject,Count,Grades,Year
0,Accounting,15.0,As,2022
1,Art and Design (Design),185.0,As,2022
2,Art and Design (Expressive),410.0,As,2022
3,Biology,570.0,As,2022
4,Business Management,110.0,As,2022


In [47]:
ind_ldf['School'] = 'Ind'
state_ldf['School'] = 'State'

ind_ldf.head()

,Subject,Count,Grades,Year,School
0,Accounting,10.0,As,2022,Ind
1,Art and Design (Design),40.0,As,2022,Ind
2,Art and Design (Expressive),65.0,As,2022,Ind
3,Biology,200.0,As,2022,Ind
4,Business Management,90.0,As,2022,Ind


In [48]:
# Merge together two df
dfc = pd.concat([state_ldf, ind_ldf])

dfc.head()

dfc.head(10)

,Subject,Count,Grades,Year,School
0,Accounting,15.0,As,2022,State
1,Art and Design (Design),185.0,As,2022,State
2,Art and Design (Expressive),410.0,As,2022,State
3,Biology,570.0,As,2022,State
4,Business Management,110.0,As,2022,State
5,Chemistry,645.0,As,2022,State
6,Chinese Languages,15.0,As,2022,State
7,Classical Studies,0.0,As,2022,State
8,Computing Science,180.0,As,2022,State
9,Design and Manufacture,20.0,As,2022,State


In [49]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2040 entries, 0 to 989
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Subject  2040 non-null   object 
 1   Count    1783 non-null   float64
 2   Grades   2040 non-null   object 
 3   Year     2040 non-null   object 
 4   School   2040 non-null   object 
dtypes: float64(1), object(4)
memory usage: 95.6+ KB


In [50]:
# Plot graph

fig = px.bar(dfc, x = "Grades", y = "Count", 
                color = "Subject",
                barmode = 'overlay',
                facet_row = 'Year',
                facet_col = 'School',
                facet_col_spacing = 0.07,
                width = 1000,
                height = 1400,
                labels={"Count": "Students"},
                category_orders = {'Grades' : ['As', 'Bs', 'Cs', 'Ds', 'NAs', 'Entries']},
                title = 'Subject grades per year at ' + level_label
                )

fig.update_yaxes(matches=None, rangemode = "tozero", showticklabels=True)

fig.write_html('./graphs/subject_bars_' + level + '.html')

fig.show()

In [51]:
# Upload to Chart Studio
py.plot(fig, filename = 'Subject_bars' + level, auto_open=True)

# Output html embed code
#tls.get_embed('https://plotly.com/~mrcrookes/14/') #change to your url

'https://plotly.com/~mrcrookes/59/'